### Build a multi-layer convolutional network

In [1]:
using TensorFlow
#using MNIST
using Distributions
using MLBase

##### prepare images list

In [41]:
train_dir="/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/train/"
test_dir="/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/test/"

images,labels=Any[],Any[]
#images,labels=Array{string,1},Array{Int32,1}
for dir in readdir(train_dir)
    files=joinpath(train_dir,dir)
    imgs=Any[]
    labs=Any[]
    for file in readdir(files)
        img=joinpath(files,file)
        imgs=vcat(imgs,img)
        labs=vcat(labs,dir)
    end
    images=vcat(images,imgs)
    labels=vcat(labels,labs)
end

lm=MLBase.labelmap(labels);
labels=MLBase.labelencode(lm,labels);

images_list=hcat(images,labels)
idx=shuffle(1:size(images_list)[1])
train_images=images_list[idx,:];


#images,labels=Array{string,1},Array{Int32,1}
images,labels=Any[],Any[]
for dir in readdir(test_dir)
    files=joinpath(test_dir,dir)
    imgs=Any[]
    labs=Any[]
    for file in readdir(files)
        img=joinpath(files,file)
        imgs=vcat(imgs,img)
        labs=vcat(labs,dir)
    end
    images=vcat(images,imgs)
    labels=vcat(labels,labs)
end

lm=MLBase.labelmap(labels);
labels=MLBase.labelencode(lm,labels);

images_list=hcat(images,labels)
idx=shuffle(1:size(images_list)[1])
test_images=images_list[idx,:];



Add `using Random` to your imports.
  likely near /home/ye/.julia/packages/IJulia/fDY4W/src/kernel.jl:38
Add `using Random` to your imports.
  likely near /home/ye/.julia/packages/IJulia/fDY4W/src/kernel.jl:38


In [44]:
function get_batch(image, label, image_W, image_H, batch_size, capacity=256,argument=true)
    images=TensorFlow.cast(image,string)
    label=TensorFlow.cast(label,Int32)
    
    # make an input queue
    input_queue=TensorFlow.train.slice_input_producer([image,label];capacity=capacity)
    label=input_queue[2]
    images_contents=TensorFlow.read_file(input_queue[1])
    image=TensorFlow.Ops.decode_jpeg(images_contents;channels=3)
    
    if argument
        image=TensorFlow.Ops.adjust_saturation(image,0.4)
        image=TensorFlow.Ops.adjust_hue(image,0.2)
        
        image=TensorFlow.image.flip_left_right(image)
        image=TensorFlow.image.flip_up_down(image)
        image=TensorFlow.Ops.resize_bilinear(image,[image_W,image_H])
    else
        image=TensorFlow.Ops.resize_bilinear(image,[image_W,image_H])
    end
    image_batch,label_batch=TensorFlow.train.batch([image,label],
                                                   batch_size=batch_size,
                                                   num_threads=64,
                                                   capacity=capacity,allow_smaller_final_batch)
    label_batch=TensorFlow.Ops.reshape(label_batch,[batch_size])
    image_batch=TensorFlow.cast(image_batch,Float32)
    return image_batch,label_batch
end

get_batch (generic function with 3 methods)

In [45]:
x=map(string,test_images[:,1])

956-element Array{String,1}:
 "/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/test/Dr_4/31659_right.jpeg"
 "/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/test/Dr_1/20309_left.jpeg" 
 "/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/test/Dr_3/21835_left.jpeg" 
 "/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/test/Dr_0/24526_left.jpeg" 
 "/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/test/Dr_0/17723_left.jpeg" 
 "/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/test/Dr_2/12650_left.jpeg" 
 "/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/test/Dr_4/7945_right.jpeg" 
 "/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/test/Dr_0/3100_right.jpeg" 
 "/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/test/Dr_4/40673_left.jpeg" 
 "/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/test/Dr_2/42753_left.jpeg" 
 "/home/ye/user/yejg/database/Kaggle_Eye/Diabetes/grades/test/Dr_0/23506_right.jpeg"
 "/home/ye/user/yejg/database/Kaggle